# **Overview**

In [7]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pandas as pd
import re
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df = pd.read_csv('/content/Data Set_csv.csv')

In [9]:
df.shape

(9176, 7)

# **Easter Eggs**

In [ ]:
!pip install --upgrade openai langchain
!pip install typing_extensions
!pip install python-dotenv
!pip install openai
from dotenv import load_dotenv
from openai import OpenAI

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
load_dotenv()

#client = OpenAI(base_url="http://localhost:4321/v1", api_key="not-needed")
client = OpenAI()

#df = pd.read_excel('data/Winter_2024_Scotia_DSD_Data_Set.xlsx')

testing_split = [
    (1, True),
]


def format_prompt(comment: str) -> str:
    prompt = f"""
    You are acting as review comment analyst for the Scotiabank Mobile Banking Application.
    Your job is to identify review comments that are external and irrelvant to the application.

    Examples of irrelevant comments:
    Lots of errors.
    Amazingly easy. I love the prompts while taking the photos and the auto shoot feature. Well done!
    I have been given excellent service, and they provide kind consideration for users.
    Worst bank on the planet. Liars, cheats, and theifs, they sell your info to scammers, they get frauded more then any other bank, do your research..

    Examples of relevant comments:
    Works well.
    Pleasant.

    Please analyze this comment to see if its relevant, and ONLY output TRUE if its relevant and FALSE if its irrelevant or if you are unsure:

    {comment}
    """
    return prompt


def llm_is_relevant(comment: str) -> bool:
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system",
             "content": "You are acting as review comment analyst for the Scotiabank Mobile Banking Application."},
            {"role": "user", "content": format_prompt(comment)}
        ],
        temperature=0,
    )
    completion_result = completion.choices[0].message.content.lower()
    if 'true' in completion_result:
        return True
    elif 'false' in completion_result:
        return False

    return False


for review_id, is_relevant in testing_split:
    comment = df.iloc[review_id]['Review']
    print(comment)
    print(review_id, llm_is_relevant(comment))

In [ ]:
def is_relevant(review):
    """
    Determine if a review is relevant to the Scotiabank Mobile Banking Application.

    :param review: A string containing the review text.
    :return: True if the review is relevant, False otherwise.
    """
    irrelevant_keywords = ["photo", "auto shoot", "hotel", "breakfast", "food options", "homie"]

    # Lowercase the review for case-insensitive matching
    review_lower = review.lower()

    # Check if any of the irrelevant keywords are in the review
    for keyword in irrelevant_keywords:
        if keyword in review_lower:
            return False

    # If none of the irrelevant keywords are found, the review is relevant
    return True


# Apply the function to the 'Review' column
df['Relevance'] = df['Review'].apply(is_relevant)

# Display the first few rows of the dataframe with the new 'Relevance' column
df.head()

# Updating the relevance status for comments like 5, 6, 7, 10, and 14
# First, find the indices of these comments in the dataframe
indices_to_update = [50, 62, 110, 157, 224]

# Update the 'Relevance' column for these indices
for index in indices_to_update:
    if index in df.index:
        df.at[index, 'Relevance'] = True

# Display the updated comments to verify the changes
df.loc[indices_to_update][['Review', 'Relevance']]


# Refining the algorithm to correctly identify relevant comments, especially those related to specific app features like cheque depositing
def refined_is_relevant(review):
    """
    Refine the relevance determination for reviews, focusing on specific app features and avoiding false negatives.

    :param review: A string containing the review text.
    :return: True if the review is relevant, False otherwise.
    """
    # List of keywords that are clearly irrelevant (not related to banking or the app)
    clearly_irrelevant_keywords = ["hotel", "breakfast", "food options", "homie", "staying", "stay at this hotel",
                                   "restaurant", "server was rude", "rooms smelled", "bathrooms were clean",
                                   "historical feel", "road trip", "hotel was clean"]

    # Lowercase the review for case-insensitive matching
    review_lower = review.lower()

    # Check if any of the clearly irrelevant keywords are in the review
    for keyword in clearly_irrelevant_keywords:
        if keyword in review_lower:
            return False

    # If none of the clearly irrelevant keywords are found, the review is considered relevant
    return True


# Apply the refined function to the 'Review' column
df['Relevance'] = df['Review'].apply(refined_is_relevant)

# Updating the relevance of previously identified borderline cases
borderline_indices = [431, 477]
for index in borderline_indices:
    if index in df.index:
        df.at[index, 'Relevance'] = True

# Display the updated comments to verify the changes
df.loc[borderline_indices][['Review', 'Relevance']]

In [ ]:
#df = pd.read_csv('/content/Updated_Comments_Classification.csv')


In [ ]:
print(df.shape)


In [ ]:
df.head()


In [ ]:
df_new = df[df['Review'].str.split().str.len() >= 3]
print(df_new.shape)

In [ ]:
df_new.head(10)

In [ ]:
df_new = df_new[df_new['Relevant'] == True]
print(df_new.shape)
df_new = df_new.drop('Relevant', axis = 1)
print(df_new.shape)

# **Desired** **Features**

We select the review with more than 50 likes

In [ ]:
df_ft = df_new

df_f = df_ft.sort_values(by=['Review_Likes'], ascending=False)

In [ ]:
df_ft = df_ft[df_ft['Review_Likes'] >= 50]

df_ft.sort_values(by=['Review_Likes'], ascending=False)

df_ft.sort_values(by=['Review_Likes'], ascending=False).to_csv('/content/desired_features.csv')

# **cleaning**

In [ ]:
df_new

In [ ]:
stop_words = set(stopwords.words('english'))
my_stop_words = {'app','scotia','scotiabank','good','bad','sucks','suck','awesome',
                 'great','worst','ok','easy','easy use','best','excellent','terrible','nice',
                 'efficient','jerk','trash','love','bank','money',
                 'big','straightforward','kinda','simple','complicated','enjoy','hate',
                 'userfriendly','user friendly','dope','banks','scotiabanks','use','time','thank',
                 'amazing','loving','cool','wow','thanks','notch','work'
                 'easier','convenient','banking','like','top','really','confusing',
                 'simple','complex','canada','canadian','easier','life','improve',
                 'horrible','planet','waste','trash','rubbish','garbage','better',
                 'fantastic','awesome','secure','find'}
stop_words = stop_words | my_stop_words


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)




def preprocess(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    # remove emoji
    text = emoji_pattern.sub(r'', text)

    # Tokenize the text
    #tokens = word_tokenize(text)

    text = text.split()

    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]

    return ' '.join(text)

In [ ]:
df_new["Review"] = df_new['Review'].apply(lambda x:preprocess(x))

In [ ]:
df_new = df_new[df_new['Review'] != '']

In [ ]:
df_new.head()

# **Sentiment**

In [ ]:
import seaborn as sns
import torch
from scipy.special import softmax
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import RobertaForSequenceClassification

tokenizer_robeta = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model_roberta = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")


In [ ]:
# example
encoded_text_ro = tokenizer_robeta('cant transfer money account either mobile app access web browser \
                                    mobile also sign scotia app cant log sign two step password',
                                   return_tensors='pt')
output_ro = model_roberta(**encoded_text_ro)
scores_ro = output_ro.logits[0].detach().numpy()
scores_ro = softmax(scores_ro)
scores_ro_dict = {
      'roberta_neg' : scores_ro[0],
      'roberta_neu' : scores_ro[1],
      'roberta_pos' : scores_ro[2]
}
print(scores_ro_dict)

In [ ]:
def sentiment_roberta(text):
  encoded_text_ro = tokenizer_robeta(text, return_tensors='pt')
  output_ro = model_roberta(**encoded_text_ro)
  scores_ro = output_ro.logits[0].detach().numpy()
  scores_ro = softmax(scores_ro)
  scores_ro_dict = {
      'roberta_neg' : scores_ro[0],
      'roberta_neu' : scores_ro[1],
      'roberta_pos' : scores_ro[2]
  }
  return scores_ro_dict

In [ ]:
df_new["index"] = np.arange(0, len(df_new))


In [ ]:
from tqdm.notebook import tqdm

result_roberta = {}
for i, row in tqdm(df_new.iterrows(), total=len(df_new)):
    try:
        text = row['Review']
        myid = row['index']
        roberta_scores = sentiment_roberta(text)
        result_roberta[myid] = roberta_scores
    except RuntimeError:
        print(f'Broke for id {myid}')

In [ ]:
df_new.index = np.arange(0, len(df_new))

In [ ]:
roberta_results_df = pd.DataFrame(result_roberta).T
roberta_results_df = roberta_results_df.reset_index().rename(columns={'index': 'Id'})
roberta_results_df = roberta_results_df.merge(df_new[["Review_ID","Date", "Rating","Review_Likes", "Review", "index"]], how = "left", left_on = "Id", right_on = "index")

In [ ]:
roberta_results_df

In [ ]:
max_sent_array = [0] * len(roberta_results_df)
negative_array = [False] * len(roberta_results_df)

for i in range(len(roberta_results_df)):
  if (roberta_results_df.loc[i, "roberta_neg"] >= roberta_results_df.loc[i, "roberta_neu"]) and (roberta_results_df.loc[i, "roberta_neg"] >= roberta_results_df.loc[i, "roberta_pos"]):
    max_sent_array[i] = 'neg'
  elif (roberta_results_df.loc[i, "roberta_neu"] >= roberta_results_df.loc[i, "roberta_neg"]) and (roberta_results_df.loc[i, "roberta_neu"] >= roberta_results_df.loc[i, "roberta_pos"]):
    max_sent_array[i] = 'neu'
  else:
    max_sent_array[i] = 'pos'
  if roberta_results_df.loc[i, "roberta_neg"] > 0.7:
    negative_array[i] = True

roberta_results_df.loc[:, "Sentiment"] = max_sent_array
roberta_results_df.loc[:, "Negative"] = negative_array

In [ ]:
roberta_results_df

# **Pain Point**

In [ ]:
df_angry = roberta_results_df[roberta_results_df['Negative'] == True]

In [ ]:
df_angry

In [ ]:
seed_topic_list_2 = [['two-step verification',
  '2sv',
  'two step',
  '2-step',
  'verification',
  'two',
  '2fa',
  'step',
  'twostep',
  'sign',],
 ['app performance',
  'performance',
  'slow',
  'fast',
  'faster',
  'slower',
  'laggy',
  'lag',
  'speed',
  'responsive',
  'unresponsive',
  'delay'],
 ['accessibility',
  'not accessible',
  'disability',
  'vision'],
 ['appointment',
  'booking',
  'book',
  'request',
  'advisor'],
 ['biometric',
  'fingerprint',
  'face id',
  'finger',
  'recognition'],
 ['budget',
  'budgeting',
  'track',
  'tracking',
  'spend'
  'spending',
  'income',
  'graph'],
 ['chat',
  'support chat',
  'customer chat',
  'chat box',
  'chatbox',
  'chat bot',
  'talk',
  'chatbot',
  'live',
  'person',
  'customer service',
  'poor service',
  'hotline',
  'hot line'],
 ['mobile deposit',
  'cheque',
  'deposit'],
 ['credit score',
  'score'],
 ['email money transfer',
  'emt',
  'e-transfer',
  'transfer',
  'email',
  'interac'],
 ['error',
  'issue',
  'problem',
  'glitch',
  'fault',
  'flaw',
  'crash',
  'glitch',
  'bug',
  'buggy',
  'bad design',
  'incorrect',
  'ui',
  'ux',
  'doesnt work',
  'navigate',
  'navigation',
  'functionality',
  'ui/ux',
  'photo',
  'picture',
  'capture',
  'unexpected',
  'unintended',
  'crash'],
 ['fee',
  'charge',
  'cost',
  'service fee',
  'charge',
  'expensive',
  'costly'],
 ['alert',
  'notification',
  'info alert',
  'warning',
  'warn',
  'notify'],
 ['international',
  'oversea',
  'overseas transfer',
  'foreign',
  'foreign transfer',
  'international',
  'transferring',
  'internationally',
  'transfer',
  'cross border',
  'border',
  'western union',
  'international money transfer'],
 ['investment',
  'invest',
  'stock',
  'portfolio',
  'gic',
  'guaranteed certificate',
  'rrsp',
  'registered retirement saving plan',
  'eft',
  'interest rate',
  'rate',
  'fund',
  'tfsa'],
 ['login',
  'logout',
  'sign in',
  'sign out',
  'log in',
  'log out',
  'logging'],
 ['quick balance',
  'fast balance',
  'balance view',
  'balance',
  'quick view'],
 ['new card',
  'request card',
  'request new card',
  'replacement',
  'replacement card',
  'lost',
  'stolen'],
 ['reward',
  'points',
  'loyalty',
  'scene',
  'scene point',
  'point',
  'shop',
  'redeem',
  'bonus'],
 ['save statement',
  'share statement',
  'statement',
  'save',
  'share',
  'transaction',
  'saving',
  'sharing',
  'history']]

In [ ]:
pain_review = df_angry['Review']

pain_review = pain_review.tolist()
pain_review = [str(pain) for pain in pain_review]

In [ ]:
!pip install bertopic

from bertopic import BERTopic

In [ ]:
!pip install hdbscan
import umap as umap

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model_pain = HDBSCAN(min_cluster_size=100, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))


umap_model_pain = umap.UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
pain_topic = BERTopic(

  # Pipeline models
  #umap_model=umap_model_pain,
  #hdbscan_model=hdbscan_model_pain,
  #vectorizer_model=vectorizer_model,

  seed_topic_list=seed_topic_list_2
)

topics_pain, probs_pain = pain_topic.fit_transform(pain_review)

In [ ]:
pain_topic.get_topic_info()

In [ ]:
pain_topic.get_topic_info().to_csv("/content/painpoint.csv")

## **Popularity with** **BERT**

In [ ]:
topic = pd.read_csv('/content/Data_Dictionary.csv')

In [ ]:
def preprocess_topics(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(token) for token in tokens]

    return tokens

In [ ]:
topic['Description'] = topic['Description'].apply(lambda x:preprocess_topics(x))

In [ ]:
topic_words_array = topic['Description'].to_numpy()

In [ ]:
seed_topic_list = [
    ["two-step verification", "2sv", "two step", "2-step", "verification"],  # 2SV
    ["app performance", "performance", "slow", "crash", "bug"],  # Application Performance
    ["accessibility", "accessible", "access"],  # Accessibility
    ["appointment", "booking", "book"],  # Appointment Booking
    ["biometric", "fingerprint", "face id", "login"],  # Biometric Login
    ["budget", "budgeting"],  # Budgeting
    ["chat", "support chat", "customer chat"],  # Chat
    ["cheque deposit", "deposit cheque", "check deposit", "mobile deposit"],  # Cheque Deposit
    ["credit score", "credit"],  # Credit Score
    ["email money transfer", "emt", "e-transfer", "transfer"],  # Email Money Transfer
    ["error", "issue", "problem", "glitch", "fault"],  # Errors
    ["fee", "charge", "cost"],  # Fee
    ["alert", "notification", "info alert"],  # Info Alerts
    ["international money", "overseas transfer", "foreign transfer"],  # International Money Movement
    ["investment", "invest", "stock", "portfolio"],  # Investments
    ["login", "logout", "sign in", "sign out", "log in", "log out"],  # Login and Logout Issues
    ["quick balance", "fast balance", "balance view"],  # Quick Balance
    ["new card", "request card", "card issue"],  # Request New Card
    ["reward", "points", "loyalty"],  # Rewards
    ["save statement", "share statement", "statement"]  # Save and Share Statements
]

In [ ]:
for i in range(20):
  seed_topic_list[i].extend(x for x in topic_words_array[i] if x not in seed_topic_list[i])

In [ ]:
docs = df_new['Review']

docs = docs.tolist()
docs = [str(doc) for doc in docs]

In [ ]:
umap_model = umap.UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)


In [ ]:
topic_model = BERTopic(seed_topic_list=seed_topic_list,umap_model=umap_model)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic_info().to_csv("/content/bert_topics.csv")

In [ ]:
# the model used for labelling
topic_model_2 = BERTopic(seed_topic_list=seed_topic_list_2,umap_model=umap_model)
topics2, probs2 = topic_model_2.fit_transform(docs)

In [ ]:
topic_model_2.get_topic_info()

In [ ]:
topic_model_2.get_topic_info().to_csv("/content/bert_topics_2.csv")

In [ ]:
topic_model.get_document_info(docs)

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))


umap_model = umap.UMAP(n_neighbors=100, n_components=5, min_dist=0.0, metric='cosine', random_state=42)


In [ ]:
seed_topic_list_3 = [['two-step verification',
  '2sv',
  'two step',
  '2-step',
  'verification',
  'two',
  '2fa',
  'step',
  'twostep',
  'sign',],
 ['app performance',
  'performance',
  'slow',
  'fast',
  'faster',
  'slower',
  'laggy',
  'lag',
  'speed',
  'responsive',
  'unresponsive',
  'delay'],
 ['accessibility',
  'accessible',
  'design',
  'not accessible',
  'disability',
  'vision'],
 ['appointment',
  'booking',
  'book',
  'request',
  'advisor'],
 ['biometric',
  'fingerprint',
  'face id',
  'finger',
  'recognition'],
 ['budget',
  'budgeting',
  'track',
  'tracking',
  'spend'
  'spending',
  'income',
  'graph'],
 ['chat',
  'support chat',
  'customer chat',
  'chat box',
  'chatbox',
  'chat bot',
  'talk',
  'chatbot',
  'live',
  'person',
  'customer service',
  'poor service',
  'hotline',
  'helpline',
  'online',
  'online customer',
  'hot line'],
 ['mobile deposit',
  'cheque',
  'deposit',
  'cheque_deposit',
  'deposit cheque',
  'capture',
  'photo'],
 ['credit score',
  'score'],
 ['email money transfer',
  'emt',
  'e-transfer',
  'transfer',
  'email',
  'interac'],
 ['error',
  'issue',
  'problem',
  'glitch',
  'fault',
  'flaw',
  'glitch',
  'bug',
  'buggy',
  'bad design',
  'incorrect',
  'ui',
  'ux',
  'doesnt work',
  'navigate',
  'navigation',
  'functionality',
  'ui/ux',
  'photo',
  'picture',
  'capture',
  'unexpected',
  'unintended',
  'crash'],
 ['fee',
  'charge',
  'cost',
  'service fee',
  'charge',
  'expensive',
  'costly'],
 ['alert',
  'notification',
  'info alert',
  'warning',
  'warn',
  'notify'],
 ['international',
  'oversea',
  'overseas transfer',
  'foreign',
  'foreign transfer',
  'international',
  'transferring',
  'internationally',
  'transfer',
  'cross border',
  'border',
  'western union',
  'international money transfer'],
 ['investment',
  'invest',
  'stock',
  'portfolio',
  'gic',
  'guaranteed certificate',
  'rrsp',
  'registered retirement saving plan',
  'eft',
  'interest rate',
  'rate',
  'fund',
  'tfsa'],
 ['login',
  'logout',
  'log',
  'logging',
  'log account',
  'sign in',
  'sign out',
  'log in',
  'log out',
  'password',
  'username',
  'logging'],
 ['quick balance',
  'fast balance',
  'balance view',
  'balance',
  'quick view'],
 ['new card',
  'request card',
  'request new card',
  'replacement',
  'replacement card',
  'lost',
  'stolen'],
 ['reward',
  'points',
  'loyalty',
  'scene',
  'scene point',
  'point',
  'shop',
  'redeem',
  'bonus'],
 ['save statement',
  'share statement',
  'statement',
  'save',
  'share',
  'transaction',
  'saving',
  'sharing',
  'payment',
  'history']]

In [ ]:
topic_model_3 = BERTopic(

  # Pipeline models
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,

  # Hyperparameters
  #top_n_words=10,
  #verbose=True

  seed_topic_list=seed_topic_list_3
)

topics_3, probs_3 = topic_model_3.fit_transform(docs)

In [ ]:
topic_model_3.get_topic_info()

In [ ]:
# df_result = pd.read_csv('/content/bert_topics.csv')

df_result = topic_model_3.get_topic_info()
df_result

In [ ]:
topic_array = [-1] * len(df_result)
topic_array_name = [''] * len(df_result)

for j in range(len(df_result)):
  topic_keyword_list = df_result.loc[j, 'Representation']
  best_match = -1
  max_matching = 0

  for i in range(20):
    cur_count = len(list(set(topic_keyword_list) & set(seed_topic_list_2[i])))

    if cur_count > max_matching:
        max_matching = cur_count
        best_match = i
  if max_matching >= 2:
    topic_array[j] = best_match
    topic_array_name[j] = topic.loc[best_match, "Topics"]


df_result.loc[:, "Topic_Category"] = topic_array
df_result.loc[:, "Topic_Category Name"] = topic_array_name

In [ ]:
df_result

In [ ]:
df_result.to_csv('/content/keyword_matching_result.csv')

In [ ]:
topics_array =  ['2SV', 'Application Performance', 'Accessibility',
       'Appointment_Booking', 'Biometric_Login', 'Budgeting', 'Chat',
       'Cheque_Deposit', 'Credit_Score', 'Email_Money_Transfer', 'Errors',
       'Fee', 'Info_Alerts', 'International_Money_Movement',
       'Investments', 'Login_and_Logout_Issues', 'Quick_Balance',
       'Request_New_Card', 'Rewards', 'Save_and_Share_Statements', 'Useless']

In [ ]:
count_array = [0] * 21

for i in range(len(df_result)):
  if df_result.loc[i, "Topic_Category"] == -1:
    count_array[20] += df_result.loc[i, "Count"]
  else:
    count_array[df_result.loc[i, "Topic_Category"]] += df_result.loc[i, "Count"]

categorizing_df = pd.DataFrame(data= {'Topic': topics_array, 'Count': count_array})

In [ ]:
categorizing_df

In [ ]:
categorizing_df.to_csv('/content/Keyword_Categorizing.csv')